In [1]:
import pickle, gzip
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import glob, os
import re, ast
import itertools
import seaborn as sns
from matplotlib import rcParams
from qiskit.visualization import plot_histogram
from matplotlib import colors
from matplotlib.colors import LinearSegmentedColormap
from seaborn import set_theme
from matplotlib import style

rcParams.update({'figure.autolayout': True})
rcParams['pdf.fonttype'] = 42
rcParams['ps.fonttype'] = 42
#rc('text', usetex=True)
rcParams['text.usetex'] = True

In [6]:
def parse_text_file_real_machine_good(filename):
    fp = open(filename,'r')
    results = dict()
    for line in fp:
        line=line.strip()
        
        #circuit: inverseQFT4 qubit: 0 theta: 0.0 phi:0.0 counts: 
        m = re.search('circuit: (.*) qubit: (.*) theta: (.*) phi:(.*) counts: ({.*})', line)
        if m:
            circuit_name = m.group(1)
            qubit = int(m.group(2))
            theta =  float(m.group(3))
            phi =  float(m.group(4))
            output = ast.literal_eval(m.group(5))
            dict_key = circuit_name
            if dict_key not in results.keys():
                # output_gold
                results[dict_key] = {'name':circuit_name, 'gate_type':'u', 'output_injections_noise':[], 'injection_qubit':[], 'theta': [], 'phi':[]}
            if (theta == 0) and (phi == 0):                
                results[dict_key]['output_gold_noise'] = output
            results[dict_key]['output_injections_noise'].append(output)
            results[dict_key]['injection_qubit'].append( qubit )
            results[dict_key]['theta'].append( theta )
            results[dict_key]['phi'].append( phi )
            
    return results

def QVF_michelson_contrast(gold_bitstring, answer, shots):    
    # Sort the answer, position 0 has the highest bitstring, position 1 the second highest
    answer_sorted = sorted(answer, key=answer.get, reverse=True)
    
    # If gold bitstring is not in answer, percentage is zero
    if gold_bitstring not in answer:
        good_percent = 0
    else:
        good_percent = answer[gold_bitstring]/shots
        
    if answer_sorted[0] == gold_bitstring: # gold bitstring has the highest count (max)
        # next bitstring is the second highest
        next_percent = answer[answer_sorted[1]]/shots 
        next_bitstring = answer_sorted[1]
    else: # gold bitstring has NOT the highest count (not max)
        next_percent = answer[answer_sorted[0]]/shots 
        next_bitstring = answer_sorted[0]
    qvf = (good_percent - next_percent) / (good_percent + next_percent)    
    return 1 - (qvf+1)/2, next_bitstring

# Read pickled data and store results in a dataframe
def build_DF_newQVFRealMachine(data):
    results = []
    shots = 1024
    gold_bitstring = max(data['output_gold_noise'], key=data['output_gold_noise'].get)#check
    original_gold_percentage = data['output_gold_noise'][gold_bitstring]/shots

    for i, answer in enumerate(data['output_injections_noise']):
        qvf, next_bitstring = QVF_michelson_contrast(gold_bitstring, answer, shots)
        max_key = max(answer, key=answer.get)
        output_percentage = answer[max_key]/shots
        next_bitstring_percentage = answer[next_bitstring]/shots
        if gold_bitstring not in answer:
            gold_percentage = 0
        else:
            gold_percentage = answer[gold_bitstring]/shots
        result = {'gold_bitstring':gold_bitstring
                , 'gold_count_percentage':gold_percentage
                , 'original_gold_count_percentage':original_gold_percentage
                , 'next_bitstring': next_bitstring
                , 'next_bitstring_percentage': next_bitstring_percentage
                , 'QVF':qvf
                , 'qubit_injected':data['injection_qubit'][i]
                , 'gate_injected':data['gate_type']
                , 'phi':data['phi'][i]
                , 'theta':data['theta'][i]
                , 'lambda':0
                , 'circuit_name':data['name']
                }
        results.append(result)
    return pd.DataFrame(results)

# Compute the new QVF for the whole circuit, as well as for each available qubit
def compute_QVF_michelson_contrast(df, circuit_name, phi, theta):
    dfFilter = df[(df.circuit_name==circuit_name) & (df.phi==phi) & (df.theta==theta)]
    #print('computing QVF for',circuit_name)
    #print(dfFilter)
    QVF = {}
    QVF['QVF_circuit'] = dfFilter['QVF'].mean()
    #print(QVF['QVF_circuit'],sum(dfFilter['success']),len(dfFilter['success']))
    #print('-----------')
    
    qubits = set(dfFilter['qubit_injected'])    
    for q in qubits:
        QVF['QVF_qubit_'+str(q)] = dfFilter[dfFilter.qubit_injected==q]['QVF'].mean()
    #QVF['gold_ok'] = dfFilter['gold_success'][0]
    return QVF

In [16]:
files_to_process = [
    '../results/Result_IBMQ_Casablanca_inverseQFT7_11-30-2021-R-20-23.txt', 
    '../results/Result_IBMQ_Jakarta_inverseQFT6_11-30-2021-R-20-23.txt', 
    '../results/Result_IBMQ_Lagos_inverseQFT4567_11-30-2021-R-20-23.txt',  # only 4 qubits
    '../results/Result_IBMQ_Perth_inverseQFT5_11-30-2021-R-20-23.txt', 
]
# read all data and insert it into one dataframe
df_newQVF_realMachine = pd.DataFrame()
for filename in files_to_process:
    data = parse_text_file_real_machine_good(filename)    
    for d in data:
        df_newQVF_realMachine = pd.concat([df_newQVF_realMachine, build_DF_newQVFRealMachine(data[d])], ignore_index=True)

In [17]:
# Get available circuits and parameters used for injection (phi and theta)
phi_list = list(set(df_newQVF_realMachine.phi))
phi_list.sort(reverse=True)
theta_list = list(set(df_newQVF_realMachine.theta))
theta_list.sort()
circuits = list(set(df_newQVF_realMachine.circuit_name))
circuits.sort()

In [18]:
results = []
for circuit in circuits:
    for phi in phi_list:
        for theta in theta_list:
            qvf = compute_QVF_michelson_contrast(df_newQVF_realMachine, circuit, phi, theta)
            qvf['circuit_name'] = circuit
            qvf['phi'] = phi
            qvf['theta'] = theta
            #qvf['threshold'] = threshold
            results.append(qvf)

new_qvfDF_noise = pd.DataFrame(results)
new_qvfDF_noise = new_qvfDF_noise[sorted(list(new_qvfDF_noise.columns))]
new_qvfDF_noise
#compute_QVF_michelson_contrast

,QVF_circuit,QVF_qubit_0,QVF_qubit_1,QVF_qubit_2,QVF_qubit_3,circuit_name,phi,theta
0,0.630638,0.560273,0.686942,0.644698,NaN,inverseQFT4,6.021386,0.000000
1,0.533835,0.599137,0.468534,NaN,NaN,inverseQFT4,6.021386,0.261799
2,0.591013,0.676236,0.505789,NaN,NaN,inverseQFT4,6.021386,0.523599
3,0.517587,0.551673,0.483502,NaN,NaN,inverseQFT4,6.021386,0.785398
4,0.537956,0.623811,0.452101,NaN,NaN,inverseQFT4,6.021386,1.047198
...,...,...,...,...,...,...,...,...
1243,0.649779,0.688347,0.651042,0.609948,NaN,inverseQFT7,0.000000,2.094395
1244,0.625205,0.596372,0.703704,0.575540,NaN,inverseQFT7,0.000000,2.356194
1245,0.628557,0.596685,0.619565,0.669421,NaN,inverseQFT7,0.000000,2.617994
1246,0.546878,0.502475,0.658273,0.479885,NaN,inverseQFT7,0.000000,2.879793


In [19]:
import warnings
warnings.filterwarnings("ignore")
#### Without multiplication sign 'x'
# theta_list_tex = ['0', '$\\frac{\pi}{12}$', '$\\frac{\pi}{6}$', '$\\frac{\pi}{4}$', '$\\frac{\pi}{3}$', '$\\frac{5\pi}{12}$', '$\\frac{\pi}{2}$', '$\\frac{7\pi}{12}$'
#               , '$\\frac{4\pi}{6}$', '$\\frac{3\pi}{4}$', '$\\frac{5\pi}{6}$', '$\\frac{11\pi}{12}$', '$\pi$']
# phi_list_tex = ['$\\frac{23\pi}{12}$', '$\\frac{11\pi}{6}$', '$\\frac{7\pi}{4}$', '$\\frac{5\pi}{3}$', '$\\frac{19\pi}{12}$'
#             , '$\\frac{6\pi}{4}$', '$\\frac{17\pi}{12}$', '$\\frac{4\pi}{3}$', '$\\frac{5\pi}{4}$', '$\\frac{7\pi}{6}$'
#             , '$\\frac{13\pi}{12}$', '$\pi$', '$\\frac{11\pi}{12}$', '$\\frac{5\pi}{6}$',  '$\\frac{3\pi}{4}$'  
#             , '$\\frac{4\pi}{6}$', '$\\frac{7\pi}{12}$', '$\\frac{\pi}{2}$', '$\\frac{5\pi}{12}$', '$\\frac{\pi}{3}$', '$\\frac{\pi}{4}$'
#             , '$\\frac{\pi}{6}$', '$\\frac{\pi}{12}$', '0']


theta_list_tex = ['0', '', '', '$\\frac{\pi}{4}$', '', '', '$\\frac{\pi}{2}$', ''
                , '', '$\\frac{3\pi}{4}$', '', '', '$\pi$']
phi_list_tex = ['', '', '$\\frac{7\pi}{4}$', '', ''
            , '$\\frac{6\pi}{4}$', '', '', '$\\frac{5\pi}{4}$', ''
            , '', '$\pi$', '', '',  '$\\frac{3\pi}{4}$'  
            , '', '', '$\\frac{\pi}{2}$', '', '', '$\\frac{\pi}{4}$'
            , '', '', '0']


dirHeatmapPlots = '../plots/real_machine/heatmaps/'
dirHistPlots = '../plots/real_machine/histograms/'
if not os.path.exists(dirHeatmapPlots):
    os.makedirs(dirHeatmapPlots)
if not os.path.exists(dirHistPlots):
    os.makedirs(dirHistPlots)    
            
for circuit in circuits:
    # get a list of qubit columns (circuit may have different number of qubits now)
    colNames = new_qvfDF_noise[new_qvfDF_noise['circuit_name']==circuit].dropna(axis=1).columns    
    QVF_list= ['QVF_circuit']
    QVF_list.extend( [x for x in colNames if re.search('QVF_qubit_.*',x)] ) # uncomment this line to include individual qubit analysis
    
    for qvf_idx in QVF_list:
        qvf_tmp = new_qvfDF_noise[new_qvfDF_noise['circuit_name']==circuit]
        qvf_tmp = qvf_tmp.pivot('phi', 'theta', qvf_idx)
        qvf_tmp.columns.name = '$\\theta$ shift'
        qvf_tmp.index.name = '$\\phi$ shift'
        fig, ax = plt.subplots(1, 1, figsize=(5, 6))
        param={'label': 'QVF'}

        divnorm = colors.TwoSlopeNorm(vmin=0, vcenter=0.5, vmax=1)
        rdgn = sns.diverging_palette(h_neg=130, h_pos=10, s=200, l=55, sep=20, as_cmap=True)
        sns.set(font_scale=1.3)
        ax = sns.heatmap(qvf_tmp, xticklabels=theta_list_tex, yticklabels=phi_list_tex, cmap="Greys", cbar_kws=param, vmin=0, vmax=1)
        fig.savefig(dirHeatmapPlots+circuit+'_'+qvf_idx+'_heatmap.png', bbox_inches='tight')
        plt.close()

        
        all_values = []
        for column in qvf_tmp:
            this_column_values = qvf_tmp[column].tolist()
            all_values += this_column_values
        one_column_df = pd.DataFrame(all_values)

        fig, ax = plt.subplots(1, 1, figsize=(6, 5))
        sns.set(font_scale=1.3)
        ax = sns.distplot(qvf_tmp, bins=256, color='black')
        plt.xlim(0, 1)

        tmp_mean = one_column_df.mean()
        tmp_stddev = one_column_df.std()
        ax.get_yaxis().set_visible(False)
        tmpFileName = dirHistPlots+circuit+'_'+qvf_idx+'_distribution_histogram_'+str(tmp_mean[0])+'_'+str(tmp_stddev[0])+'.pdf'
        fig.savefig(tmpFileName, bbox_inches = 'tight')
        plt.close()

/home/dagoliveira/.local/lib/python3.9/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/home/dagoliveira/.local/lib/python3.9/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibility) or `histplot` (an axes-level function for histograms).
  warnings.warn(msg, FutureWarning)
/home/dagoliveira/.local/lib/python3.9/site-packages/seaborn/distributions.py:2557: FutureWarning: `distplot` is a deprecated function and will be removed in a future version. Please adapt your code to use either `displot` (a figure-level function with similar flexibili

In [20]:
# Plot delta heatmaps
dirPlots = '../plots/real_machine/deltaHeatmaps/'
if not os.path.exists(dirPlots):
    os.makedirs(dirPlots)

for circuit in circuits:
    # get a list of qubit columns (circuit may have different number of qubits now)
    colNames = new_qvfDF_noise[new_qvfDF_noise['circuit_name']==circuit].dropna(axis=1).columns    
    QVF_list =  [x for x in colNames if re.search('QVF_qubit_.*',x)]  # uncomment this line to include individual qubit analysis    
    
    for pair in itertools.combinations(QVF_list, 2):
            qvf_tmp = new_qvfDF_noise[new_qvfDF_noise['circuit_name']==circuit].copy()
            qvf_tmp['delta'] = qvf_tmp[pair[0]] - qvf_tmp[pair[1]]        
            qvf_tmp = qvf_tmp.pivot('phi', 'theta', 'delta')
            qvf_tmp.columns.name = '$\\theta$ shift'
            qvf_tmp.index.name = '$\\phi$ shift'
            fig, ax = plt.subplots(1, 1, figsize=(5, 6))
            label = '$\Delta$QVF = '+pair[0].replace('_','')+' - '+pair[1].replace('_','')
            param={'label': label}
            ax = sns.heatmap(qvf_tmp, xticklabels=theta_list_tex, yticklabels=phi_list_tex, cmap='seismic', cbar_kws=param, vmin=-1, vmax=1)            
            plt.axhline(y=20.5, color="blue", linestyle="--")       #T at phi=pi/4
            plt.text(13.25, 20.7, r'$T$', fontsize=10, color="blue")            
            plt.axhline(y=17.5, color="black", linestyle="--")      #S at phi=pi/2  
            plt.text(13.25, 17.7, r'$S$', fontsize=10, color="black")       
            plt.axhline(y=11.5, color="cyan", linestyle="--")       #Z at phi=pi   
            plt.text(13.25, 11.7, r'$Z$', fontsize=10, color="cyan")          
            plt.axvline(x=12.5, color="purple", linestyle="--")     #X,Y at theta=pi
            plt.text(12, -0.5, r'$X, Y$', fontsize=10, color="purple")
            fig.savefig(dirPlots+circuit+'_'+pair[0]+'-'+pair[1]+'_delta_heatmap.pdf', bbox_inches='tight')
            plt.close()
